**Mount Drive**

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


**Function and library loading**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle  # Import the shuffle function
import cv2
from tensorflow.keras.applications import ResNet50


# Function to load images from folder and apply preprocessing
def load_images_from_folder(folder_path):
    images = []
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = cv2.resize(img, (224, 224))  # Resize to (224, 224)
        images.append(img)
    return np.array(images)






**Preparing dataset to use**

In [ ]:
# Path to the dataset
training_folder_path = '/content/drive/MyDrive/BoneFractureDataset/archive/train'
testing_folder_path = '/content/drive/MyDrive/BoneFractureDataset/archive/val'

# load fractured and not fractured images from training and testing sets
train_fractured_images = load_images_from_folder(os.path.join(training_folder_path, 'fractured'))
train_not_fractured_images = load_images_from_folder(os.path.join(training_folder_path, 'not fractured'))

test_fractured_images = load_images_from_folder(os.path.join(testing_folder_path, 'fractured'))
test_not_fractured_images = load_images_from_folder(os.path.join(testing_folder_path, 'not fractured'))

# combine fractured images
all_fractured_images = np.concatenate((train_fractured_images, test_fractured_images), axis=0)
all_fractured_labels = np.ones(all_fractured_images.shape[0])

# combine not fractured images
all_not_fractured_images = np.concatenate((train_not_fractured_images, test_not_fractured_images), axis=0)
all_not_fractured_labels = np.zeros(all_not_fractured_images.shape[0])

# combine all images and labels and shuffle the data
all_images = np.concatenate((all_fractured_images, all_not_fractured_images), axis=0)
all_labels = np.concatenate((all_fractured_labels, all_not_fractured_labels), axis=0)
all_images, all_labels = shuffle(all_images, all_labels, random_state=42)

# split the shuffled data into training, validation, and testing sets (60% - 20% - 20% split)
train_images, temp_images, train_labels, temp_labels = train_test_split(all_images, all_labels, test_size=0.4, random_state=42)
valid_images, test_images, valid_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.5, random_state=42)


**Use ResNet50 pretrained on ImageNet for transfer learning**

In [ ]:
# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# custom classification layers added on top of pretrained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

94765736/94765736 [==============================] - 5s 0us/step


**train the model**

In [ ]:
# Train the model with validation data
model.fit(train_images, train_labels, validation_data=(valid_images, valid_labels), epochs=20, batch_size=128)

# Evaluate the model on the test data
predictions = model.predict(test_images)
rounded_predictions = np.round(predictions).flatten().astype(int)
accuracy = accuracy_score(test_labels, rounded_predictions)
print("Test Accuracy:", accuracy)


Epoch 1/20
45/45 [==============================] - 49s 644ms/step - loss: 0.6661 - accuracy: 0.6373 - val_loss: 0.4896 - val_accuracy: 0.7802
Epoch 2/20
45/45 [==============================] - 20s 452ms/step - loss: 0.4337 - accuracy: 0.7958 - val_loss: 0.3545 - val_accuracy: 0.8595
Epoch 3/20
45/45 [==============================] - 26s 578ms/step - loss: 0.3174 - accuracy: 0.8679 - val_loss: 0.2446 - val_accuracy: 0.9192
Epoch 4/20
45/45 [==============================] - 26s 583ms/step - loss: 0.2527 - accuracy: 0.9022 - val_loss: 0.2041 - val_accuracy: 0.9319
Epoch 5/20
45/45 [==============================] - 26s 587ms/step - loss: 0.2025 - accuracy: 0.9267 - val_loss: 0.1522 - val_accuracy: 0.9599
Epoch 6/20
45/45 [==============================] - 26s 582ms/step - loss: 0.1650 - accuracy: 0.9459 - val_loss: 0.1557 - val_accuracy: 0.9445
Epoch 7/20
45/45 [==============================] - 26s 584ms/step - loss: 0.1302 - accuracy: 0.9614 - val_loss: 0.1021 - val_accuracy: 0.9731

**Results**

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
import cv2
import os
# Print confusion matrix
confusion = confusion_matrix(test_labels, rounded_predictions)
print("Confusion Matrix:")
print(confusion)


report = classification_report(test_labels, rounded_predictions)
print("Classification Report:\n", report)



Confusion Matrix:
[[922   2]
 [ 12 957]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       924
         1.0       1.00      0.99      0.99       969

    accuracy                           0.99      1893
   macro avg       0.99      0.99      0.99      1893
weighted avg       0.99      0.99      0.99      1893



**Save the model**

In [ ]:
# Save the trained model to the specified location
save_path = '/content/drive/MyDrive/BoneFractureDataset/archive/pretrainedResmodelBoneFractureClassifier2.h5'
model.save(save_path)
